In [7]:
# load package
import numpy as np
import pandas as pd
import pickle
import xlearn as xl
import logging

# set path
import os
default_path = "/Users/francislin/temp/"
os.chdir(default_path)

# skip interaction warning
import warnings; 
warnings.filterwarnings('ignore')
#你可以选择修改 ast_note_iteractively kernel 选项来使得 Jupyter 为每一行的变量或语句执行这个操作，以便你可以立即看到多条语句一起输出
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
with open('all_data.pickle','rb') as f:
    df= pickle.load(f)

In [4]:
df.head()

,cuisine,id,used_pepper,used_salt,used_oil,used_garlic,used_ground,used_fresh,used_sauce,used_sugar,...,used_hock,used_rose,used_ciabatta,used_ready,used_regular,used_glass,used_safflower,used_jeera,used_Oil,used_oven
0,greek,10259,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,southern_us,25693,1,1,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,filipino,20130,1,1,1,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,indian,22213,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,indian,13162,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
# 區分train test
dataTrain = df[pd.notnull(df['cuisine'])]
dataTest = df[~pd.notnull(df['cuisine'])]
dataTrain.columns

Index(['cuisine', 'id', 'used_pepper', 'used_salt', 'used_oil', 'used_garlic',
       'used_ground', 'used_fresh', 'used_sauce', 'used_sugar',
       ...
       'used_hock', 'used_rose', 'used_ciabatta', 'used_ready', 'used_regular',
       'used_glass', 'used_safflower', 'used_jeera', 'used_Oil', 'used_oven'],
      dtype='object', length=902)

In [10]:
data_key=df[['id','cuisine']]
#data = df.drop(['id','cuisine'],axis=1)

In [5]:
fm_model = xl.FMModel(task='binary', init=0.1, epoch=10, k=10, lr=0.1, reg_lambda=0.1, opt='sgd', metric='auc')

In [6]:
fm_model

FMModel(alpha=1, beta=1, block_size=500, epoch=10, fold=1, init=0.1, k=10,
    lambda_1=1, lambda_2=1, lr=0.1, metric='auc', model_type='fm',
    n_jobs=4, nthread=None, opt='sgd', reg_lambda=0.1, stop_window=2,
    task='binary')

In [40]:
def cuisine_embedding_fm(emb_cuisine_input, cuisine_dummies, cuisine_dummies_train, fm_model, cuisine, id):
    #, label, train_id
    logging.info('Fit the FM model and derive the embedding values')
    fm_model.fit(cuisine_dummies_train, emb_cuisine_input.loc[id, cuisine])
    emb_weight = fm_model.weights[1]
    cuisine_emb = np.matmul(cuisine_dummies.values, emb_weight)
    #cuisine_emb = pd.DataFrame(cuisine_emb, columns=['cuisine'+edu_state[state]+str(i) for i in range(emb_weight.shape[1])])
    
    return emb_weight, cuisine_emb

In [41]:
emb_cuisine_input = df[['id','cuisine']]#.reset_index(drop=True)
cuisine_dummies = df.drop(['id','cuisine'],axis=1)
cuisine_dummies_train = dataTrain.drop(['id','cuisine'],axis=1)
cuisine = df.cuisine
id = df.id

In [42]:
cuisine_embedding_fm(emb_cuisine_input, cuisine_dummies, cuisine_dummies_train, fm_model, cuisine, id)

ValueError: Location based indexing can only have [labels (MUST BE IN THE INDEX), slices of labels (BOTH endpoints included! Can be slices of integers if the index is integers), listlike of labels, boolean] types

In [44]:
emb_cuisine_input.loc[id, cuisine]

ValueError: Location based indexing can only have [labels (MUST BE IN THE INDEX), slices of labels (BOTH endpoints included! Can be slices of integers if the index is integers), listlike of labels, boolean] types